# Calculating Network Error with Loss

## Categorical Cross-Entropy Loss

![Categorical Cross-Entropy Loss](./categorical-ce-loss.png)

Example:

In [1]:
import math

softmax_output = [0.7, 0.1, 0.2]
target_output = [1,0,0]

loss = -(math.log(softmax_output[0]) * target_output[0] +
         math.log(softmax_output[1]) * target_output[1] +
         math.log(softmax_output[2]) * target_output[2])

print(loss)

0.35667494393873245
